** Importing the necessary modules **

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from wordcloud import WordCloud
from folium.plugins import HeatMap
import folium
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


** Importing and exploring the dataset **

In [ ]:
data=pd.read_csv('../input/zomato.csv')
data.info()

In [ ]:
data.head()

Now, we drop the unnecessary features like URL, name and phone

In [ ]:
data.drop('url',axis=1,inplace=True)
data.drop('phone',axis=1,inplace=True)

In [ ]:
data.tail()

** Cleaning the dataset**

Replace the unknown rating i.e "-","NEW" to null.

In [ ]:
data['rate'] = data['rate'].replace('NEW',np.NaN)
data['rate'] = data['rate'].replace('-',np.NaN)
data.dropna(how = 'any', inplace = True)

In [ ]:
data['rate'] = data.loc[:,'rate'].replace('[ ]','',regex = True)
data['rate'] = data['rate'].astype(str)
data['rate'] = data['rate'].apply(lambda r: r.replace('/5',''))
data['rate'] = data['rate'].apply(lambda r: float(r))

In [ ]:
data['approx_cost(for two people)'] = data['approx_cost(for two people)'].str.replace(',','')
data['approx_cost(for two people)'] = data['approx_cost(for two people)'].astype(int)

In [ ]:
data.head()

In [ ]:
loc=data.loc[:, "location"].unique()
print(loc)
loc.sort()

In [ ]:
def show_articles_more_than(Restaurant_Name=''):
    return data[data['name'].str.contains(Restaurant_Name)]

In [ ]:
def show_Restaurants_according_to_search(Location=loc,
                                         Restaurant_Type=['Buffet', 
                                             'Cafes',
                                             'Delivery',
                                             'Desserts',
                                             'Dine-out',
                                             'Drinks & nightlife',
                                             'Pubs and bars'],
                            Min_Rating=(0,5,0.1),
                            Max_Cost_For_Two_People=(100,5000,50)):
    print("")
    return data[ (data['rate'] > Min_Rating) 
                &(data['listed_in(type)'] == Restaurant_Type) 
                &(data['location'] == Location) 
                & (data['approx_cost(for two people)'] < Max_Cost_For_Two_People)]

** Top 10 resturants chians in Bangaluru **

In [ ]:
plt.figure(figsize=(10,7))
chains=data['name'].value_counts()[:10]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants chains in Bangaluru")
plt.xlabel("Number of outlets")

Resturants delivering order online 

In [ ]:
sns.countplot(x=data['online_order'])
fig = plt.gcf()
fig.set_size_inches(6,6)
plt.title('Restaurants delivering order online')

** Types of the orders getting online. **

In [ ]:
sns.countplot(x=data['online_order'],hue=data['listed_in(type)'])
fig=plt.gcf()
fig.set_size_inches(14,8)


** Resturants giving booking table facility. **

In [ ]:
sns.countplot(x=data['book_table'])
fig = plt.gcf()
fig.set_size_inches(6,6)
plt.title('Restaurants providing Table booking facility:')

** Dish types ordered by the booked tables. **

In [ ]:
sns.countplot(x=data['book_table'],hue=data['listed_in(type)'])
fig=plt.gcf()
fig.set_size_inches(14,8)


** Most common resturants **

In [ ]:
print("All different dining type restaurants")
data['listed_in(type)'].unique()

In [ ]:
plt.figure(figsize=(20,7))
sns.countplot(x=data['rest_type'],hue=data['listed_in(type)'])
plt.title("Restaurant types")
plt.xlabel("count")

** Rating of the resturants. **

In [ ]:
data.rate.unique()

** Rating distribution. **

In [ ]:
sns.countplot(x=data['rate'])
fig = plt.gcf()
fig.set_size_inches(20,6)
plt.title('Rating of the resturants')

** Percentage of the ratings of the resturants. **

In [ ]:
slices=[((data.rate>=0) & (data.rate<1)).sum(),
        ((data.rate>=1) & (data.rate<2)).sum(),
        ((data.rate>=2) & (data.rate<3)).sum(),
        ((data.rate>=3.0) & (data.rate<4)).sum(),
        ((data.rate>=4) & (data.rate<5)).sum(),
       ]
labels=['0-1','1-2','2-3','3-4','4-5']
plt.pie(slices, labels=labels, autopct='%1.0f%%', pctdistance=.5, labeldistance=1.2,shadow=True)
fig = plt.gcf()
plt.title("Percentage of restaurants according to their ratings")

fig.set_size_inches(10,10)
plt.show()

** Rating of the resturants vs online delivery **

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.countplot(x='rate',hue='online_order',data=data)
plt.title('Rating of Restaurants vs Online Delivery')
plt.show()

** Rating of the distribution vs Table Booking **

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.countplot(x='rate',hue='book_table',data=data)
plt.title('Rating of Restaurants vs Table Booking')
plt.show()

** Votes vs Rating **

In [ ]:
data.plot(kind='scatter',x='rate',y='votes',marker='o',color='g',grid=True,figsize=(20,7))
plt.title('Votes Versus Rate');

** Votes vs Cost for the two persons. **

In [ ]:
data.plot(kind='scatter',x='rate',y='approx_cost(for two people)',marker='o',color='b',grid=True,figsize=(20,7))
plt.title('Average cost for Two Persons Versus Rate',weight='bold');

In [ ]:
print("all different cuisines:")
cuisines = set()
for i in data['cuisines']:
    for j in str(i).split(', '):
        cuisines.add(j)
cuisines

In [ ]:
import re
data=data[data['dish_liked'].notnull()]
data.index=range(data.shape[0])
likes=[]
for i in range(data.shape[0]):
    splited_array=re.split(',',data['dish_liked'][i])
    for item in splited_array:
        likes.append(item)


** Count of the most liked dishes of Bengaluru. **

In [ ]:
print("Count of Most liked dishes of Bangalore")
favourite_food = pd.Series(likes).value_counts()
favourite_food.head(20)

** Most popular cuisines in Bangaluru **

In [ ]:
plt.figure(figsize=(7,7))
cuisines=data['cuisines'].value_counts()[:10]
sns.barplot(cuisines,cuisines.index,palette='rocket')
plt.xlabel('Count')
plt.title("Most popular cuisines of Bangalore")

In [ ]:
locCount=data['location'].value_counts().sort_values(ascending=True)
locCount

In [ ]:
CityCount=data['listed_in(city)'].value_counts().sort_values(ascending=True)
CityCount

** Top voted resturants **

In [ ]:
plt.figure(figsize=(20,7))
data.groupby('location')['votes'].sum().sort_values(ascending=False)[:10].plot('bar',color='r')
plt.title('Top 10 Voted neighbourhoods',weight='bold')
plt.ylabel('Count');
